In [9]:
# !pip install bayesian-optimization

In [2]:
# ML
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from bayes_opt import BayesianOptimization
import xgboost as xgb
import pickle

In [3]:
df = pd.read_csv("pages/Microbiota_composition.csv")
# df = df.drop("country", axis=1)

In [5]:
df.head()

,condition,Streptococcus_anginosus___ref_mOTU_v2_0004__,Enterobacteriaceae_sp.___ref_mOTU_v2_0036__,Citrobacter_sp.___ref_mOTU_v2_0076__,Klebsiella_michiganensis/oxytoca___ref_mOTU_v2_0079__,Enterococcus_faecalis___ref_mOTU_v2_0116__,Lactobacillus_salivarius___ref_mOTU_v2_0125__,Dielma_fastidiosa___ref_mOTU_v2_0138__,Streptococcus_constellatus/intermedius___ref_mOTU_v2_0143__,Streptococcus_parasanguinis___ref_mOTU_v2_0144__,...,unknown_Clostridiales___meta_mOTU_v2_7778__,unknown_Clostridiales___meta_mOTU_v2_7781__,unknown_Clostridiales___meta_mOTU_v2_7782__,unknown_Clostridiales___meta_mOTU_v2_7784__,Clostridium_sp._CAG__230___meta_mOTU_v2_7788__,Clostridium_sp._CAG__1193___meta_mOTU_v2_7789__,unknown_Erysipelotrichaceae___meta_mOTU_v2_7790__,unknown_Clostridiales___meta_mOTU_v2_7795__,unknown_Clostridiales___meta_mOTU_v2_7800__,country
0,control,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000083,0.000000,0.000249,...,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.000083,0.000083,Austria
1,control,0.000591,0.000066,0.000000,0.000000,0.000000,0.000000,0.000000,0.000066,0.002102,...,0.000000,0.000000,0.000788,0.0,0.00000,0.0,0.0,0.000000,0.000000,Austria
2,control,0.000840,0.000065,0.002454,0.000000,0.000065,0.013111,0.000000,0.000065,0.012013,...,0.000000,0.000194,0.000065,0.0,0.00084,0.0,0.0,0.001808,0.000000,Austria
3,control,0.000000,0.000071,0.000000,0.000000,0.000000,0.000000,0.000213,0.000071,0.000142,...,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.000000,0.000000,Austria
4,control,0.000085,0.019236,0.001111,0.000256,0.000085,0.000085,0.000000,0.000000,0.000513,...,0.000171,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.000000,0.000000,Austria


In [12]:
# Features
X = df.drop(columns=["condition", "country"], axis=1)

# Target
y = df["condition"]

# lambda to scale the target into numeric (CRC : 1, control : 0)
y = y.apply(lambda x: 1 if x == "CRC" else 0)

# Split df into train and test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Boolean filter to keep numeric column
numeric_features = df.select_dtypes(include=['number']).columns
numeric_transformer = Pipeline(steps=[("scaler", StandardScaler())])

# Selection of the categorical feature to preprocess
# categorical_feature = X.select_dtypes(exclude=['number']).columns
# feature_transformer = Pipeline(steps=[("encoder", OneHotEncoder(drop="first"))])

# Use ColumnTransformer to make a preprocessor object that describes all the treatments to be done
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        # ("cat", feature_transformer, categorical_feature),
    ]
)

# Prétraitement des données (X_train, X_test) avec une normalisation
preprocessor.fit(X_train)

# Spécifiez le nom du fichier dans lequel vous souhaitez enregistrer l'objet
scaler_name = "scaler.pkl"

# Utilisez la bibliothèque pickle pour enregistrer l'objet scaler dans le fichier
with open(scaler_name, 'wb') as fichier:
    pickle.dump(preprocessor, fichier)

X_train = preprocessor.transform(X_train)
X_test = preprocessor.transform(X_test)
# input_shape = X_train_scaled.shape[1]

# Définir la fonction d'évaluation
def evaluate_xgb(n_estimators, max_depth, learning_rate, gamma, min_child_weight, alpha, reg_lambda, eta):
    model = xgb.XGBClassifier(
        n_estimators=int(n_estimators),
        max_depth=int(max_depth),
        learning_rate=learning_rate,
        gamma=gamma,
        objective='binary:logistic',
        min_child_weight=min_child_weight,
        random_state=42,
        alpha=alpha,
        reg_lambda=reg_lambda,
        booster='gbtree',
        eta=eta
        )

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

# Définir l'espace de recherche
pbounds = {
    'max_depth': (3, 10),
    'learning_rate': (0.001, 0.1),
    'n_estimators': (100, 200),
    'gamma': (0, 5.0),
    'min_child_weight': (0, 10.0),
    'alpha': (0, 2),
    'reg_lambda': (0, 2),
    'eta': (0, 1)
    }

# Initialiser l'optimiseur bayésien
optimizer = BayesianOptimization(f=evaluate_xgb, pbounds=pbounds, random_state=42)

# Lancer l'optimisation
optimizer.maximize(init_points=10, n_iter=40) # 5 points aléatoires, 30 itérations

# Récupérer les hyperparamètres optimaux
best_params = optimizer.max['params']

# Entraîner le modèle final avec les meilleurs hyperparamètres
best_model = xgb.XGBClassifier(n_estimators=int(best_params['n_estimators']),
                           max_depth=int(best_params['max_depth']),
                           learning_rate=best_params['learning_rate'],
                           gamma=best_params['gamma'],
                           min_child_weight=best_params['min_child_weight'],
                           alpha=best_params['alpha'],
                           reg_lambda=best_params['reg_lambda'],
                           eta=best_params['eta']
                           )

best_model.fit(X_train, y_train)

# Enregistrez le modèle dans un fichier pickle 
with open("xgboost_classifier_model.pkl", "wb") as file:
    pickle.dump(best_model, file)

# Charger le modèle depuis le fichier pickle
with open("xgboost_classifier_model.pkl", "rb") as file:
    loaded_model = pickle.load(file)

# Make predictions on the test set
# predictions = best_model.predict(X_test)

# Utilisez le modèle chargé pour faire des prédictions, par exemple
predictions = loaded_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy:.4f}")

|   iter    |  target   |   alpha   |    eta    |   gamma   | learni... | max_depth | min_ch... | n_esti... | reg_la... |
-------------------------------------------------------------------------------------------------------------------------
| 1         | 0.7792    | 0.7491    | 0.9507    | 3.66      | 0.06027   | 4.092     | 1.56      | 105.8     | 1.732     |
| 2         | 0.7857    | 1.202     | 0.7081    | 0.1029    | 0.09702   | 8.827     | 2.123     | 118.2     | 0.3668    |
| 3         | 0.7597    | 0.6085    | 0.5248    | 2.16      | 0.02983   | 7.283     | 1.395     | 129.2     | 0.7327    |
| 4         | 0.7987    | 0.9121    | 0.7852    | 0.9984    | 0.05191   | 7.147     | 0.4645    | 160.8     | 0.341     |
| 5         | 0.7922    | 0.1301    | 0.9489    | 4.828     | 0.08103   | 5.132     | 0.9767    | 168.4     | 0.8803    |
| 6         | 0.7662    | 0.2441    | 0.4952    | 0.1719    | 0.09102   | 4.811     | 6.625     | 131.2     | 1.04      |
| 7         | 0.7597    